This tutorial can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2025.2_tutorials/wallaroo-run-anywhere/inference/arm/wallaroo-arm-byop-vgg16).

## Model Upload API Error Message Tests


## Tutorial Steps

### Import Libraries

The first step is to import the libraries we'll be using.  These are included by default in the Wallaroo instance's JupyterHub service.

In [41]:
import numpy as np
import pandas as pd
import pyarrow as pa
import wallaroo

from wallaroo.deployment_config import DeploymentConfigBuilder
from wallaroo.framework import Framework
import base64
import requests
import time
import json

### Open a Connection to Wallaroo

The next step is connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more details on logging in through Wallaroo, see the [Wallaroo SDK Essentials Guide: Client Connection](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [42]:
wl = wallaroo.Client()



In [43]:
wl.api_endpoint

'https://autoscale-uat-gcp.wallaroo.dev'

### Set Variables

We'll set the name of our workspace, pipeline, models and files.  Workspace names must be unique across the Wallaroo workspace.  For this, we'll add in a randomly generated 4 characters to the workspace name to prevent collisions with other users' workspaces.  If running this tutorial, we recommend hard coding the workspace name so it will function in the same workspace each time it's run.



In [44]:
workspace_name = f'vgg16-clustering-workspace-error-api'


### Create Workspace and Pipeline

We will now create the Wallaroo workspace to store our model and set it as the current workspace.  Future commands will default to this workspace for pipeline creation, model uploads, etc.  We'll create our Wallaroo pipeline that is used to deploy our arbitrary Python model.

In [45]:
workspace = wl.get_workspace(name=workspace_name, create_if_not_exist=True)
wl.set_current_workspace(workspace)


{'name': 'vgg16-clustering-workspace-error-api', 'id': 1814, 'archived': False, 'created_by': 'john.hummel@wallaroo.ai', 'created_at': '2025-12-18T21:54:59.44398+00:00', 'models': [{'name': 'api-byop-error-requirements-empty', 'versions': 1, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 12, 23, 21, 12, 57, 876767, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 12, 23, 21, 12, 57, 876767, tzinfo=tzutc())}, {'name': 'api-byop-error-demo-unmodified', 'versions': 2, 'owner_id': '""', 'last_update_time': datetime.datetime(2025, 12, 23, 20, 54, 20, 692928, tzinfo=tzutc()), 'created_at': datetime.datetime(2025, 12, 19, 2, 0, 53, 509654, tzinfo=tzutc())}], 'pipelines': [{'name': 'vgg16-clustering-error-tests', 'create_time': datetime.datetime(2025, 12, 18, 21, 54, 59, 717343, tzinfo=tzutc()), 'definition': '[]'}]}

Define standard parameters

In [46]:
input_schema = pa.schema([
    pa.field('images', pa.list_(
        pa.list_(
            pa.list_(
                pa.int64(),
                list_size=3
            ),
            list_size=32
        ),
        list_size=32
    )),
])

output_schema = pa.schema([
    pa.field('predictions', pa.int64()),
])

encoded_input_schema = base64.b64encode(
                bytes(input_schema.serialize())
            ).decode("utf8")

encoded_output_schema = base64.b64encode(
                bytes(output_schema.serialize())
            ).decode("utf8")

framework = 'custom'

### Test 1:  Normal Model

Unmodified model to show it can upload successfully without issue.



In [47]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-demo-unmodified'
model_file_name = './models/byop-vgg-unmodified.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-demo-unmodified.'

{'insert_models': {'returning': [{'models': [{'id': 1141}]}]}}

In [ ]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": 1139
}

status = "pending_load_container"

while (status == 'pending_load_container'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

{'model_version': {'model_version': {'name': 'api-byop-error-demo-unmodified',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg-unmodified.zip/model'}}},
   'id': 1139,
   'image_path': 'proxy.replicated.com/proxy/wallaroo/ghcr.io/wallaroolabs/mac-deploy:v2025.2.0-6480',
   'status': 'ready',
   'task_id': '66cf4101-c153-40d6-ab65-c0bda54ffb3c',
   'file_info': {'version': '66a8cdb4-d054-4194-a4ea-c024cf172e61',
    'sha': '79472d4b9652937c15e1887478c23effe1160363b985920e083fec72868810f4',
    'file_name': './models/byop-vgg-unmodified.zip',
    'size': 54695306},
   'created_on_version': '2025.2.0',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T20:54:20.692928+00:00',
   'deployed': False,
   'error_summary': None},
  'config': {'id': 1710,
   'model_version_id': 113

### Test 2:  Requirements.txt is missing

In the following test, the Requirements.txt file is empty.

zip file contents:

```bash
Archive:  byop-vgg16-requirements-empty.zip
  inflating: custom_inference.py     
  inflating: feature_extractor.h5    
  inflating: kmeans.pkl      
```


In [49]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-requirements-empty'
model_file_name = './models/byop-vgg16-requirements-empty.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}



headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-requirements-empty.'

{'insert_models': {'returning': [{'models': [{'id': 1142}]}]}}

In [50]:
requirements_empty_id = response['insert_models']['returning'][0]['models'][0]['id']

In [54]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": requirements_empty_id
}

status = "pending_load_container"

while (status == 'pending_load_container'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'pending_load_container'

'pending_load_container'

'pending_load_container'

'pending_load_container'

'pending_load_container'

...

'pending_load_container'

'pending_load_container'

'pending_load_container'

'error'

{'model_version': {'model_version': {'name': 'api-byop-error-requirements-empty',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-vgg16-requirements-empty.zip/model'}}},
   'id': 1142,
   'image_path': None,
   'status': 'error',
   'task_id': '66685188-2d69-46f8-8596-e3197c19481d',
   'file_info': {'version': '882a59bd-799c-4828-a789-39e8b1fa8f74',
    'sha': '47e1962acba0a53481d9e3929265633f3e8fc0e7b067a63b1e79140d94f3a8a8',
    'file_name': './models/byop-vgg16-requirements-empty.zip',
    'size': 54694320},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T21:43:51.193448+00:00',
   'deployed': False,
   'error_summary': 'Framework.CUSTOM requires a requirements.txt to be included at the root level within the uploaded .zip file'},
  '

### Test 3: A .py file without an Inference class 



In [64]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'api-byop-error-inference-missing'
model_file_name = './models/byop-llamacpp-infbuild-no-inf.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}



headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: api-byop-error-inference-missing.'

{'insert_models': {'returning': [{'models': [{'id': 1145}]}]}}

In [65]:
model_missing_inference = response['insert_models']['returning'][0]['models'][0]['id']

In [67]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_missing_inference
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'api-byop-error-inference-missing',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-infbuild-no-inf.zip/model'}}},
   'id': 1145,
   'image_path': None,
   'status': 'error',
   'task_id': 'bb07e7a7-bc8e-4123-b8ab-927fd1ff932a',
   'file_info': {'version': '462cdd79-3524-4ad8-9df1-92cb8a21a4c8',
    'sha': 'f8d24672bb026a7d2cca8a94bbdfb58cc99b631ab6238bd3f27969578381fc29',
    'file_name': './models/byop-llamacpp-infbuild-no-inf.zip',
    'size': 1090539},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T21:59:01.568287+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1716,
   'model_version_id': 1145,
   'runtime': 'flight',
   'filter_threshold': None,
 

### Test 4: An Inference class without expected_model_types method

In [68]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'no-expected-model-types'
model_file_name = './models/byop-llamacpp-inf-no-exp-model-types.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: no-expected-model-types.'

{'insert_models': {'returning': [{'models': [{'id': 1147}]}]}}

In [69]:
missing_expected_model_types = response['insert_models']['returning'][0]['models'][0]['id']

In [71]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": missing_expected_model_types
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'no-expected-model-types',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-inf-no-exp-model-types.zip/model'}}},
   'id': 1147,
   'image_path': None,
   'status': 'error',
   'task_id': '1d71d523-e4e4-4e53-8acc-58161e3cf000',
   'file_info': {'version': 'c1740289-20ca-4c04-9604-660cf61781c9',
    'sha': 'b9b96606a05baf8c5d52594f688c1a85cdf9e97c39d0b60cc9b11e35563045a4',
    'file_name': './models/byop-llamacpp-inf-no-exp-model-types.zip',
    'size': 1090626},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:02:07.988902+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1718,
   'model_version_id': 1147,
   'runtime': 'flight',
   'filter_threshold': No

### Test 5: An Inference class without a predict method


In [72]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'missing-precict'
model_file_name = './models/byop-llamacpp-inf-no-predict.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)

'Uploaded Model Name: missing-precict.'

{'insert_models': {'returning': [{'models': [{'id': 1148}]}]}}

In [73]:
model_version_id_missing_predict = response['insert_models']['returning'][0]['models'][0]['id']

In [74]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_version_id_missing_predict
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

...

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'missing-precict',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-inf-no-predict.zip/model'}}},
   'id': 1148,
   'image_path': None,
   'status': 'error',
   'task_id': 'a8860a25-3326-4ffb-b65b-4a258b8e1f83',
   'file_info': {'version': '1da190b4-e1c6-4ae3-87d9-944a767cb7d9',
    'sha': '5314c3da35aa67f29ddc15af123ff85d490f97f90e4e0451b934e78b403ced58',
    'file_name': './models/byop-llamacpp-inf-no-predict.zip',
    'size': 1090270},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:07:21.369994+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1719,
   'model_version_id': 1148,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': 

###  Test 5: A Predict method that does not accept a dictionary of numpy arrays

In [78]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-no-accept-numpy'
model_file_name = './models/byop-no-accept-numpy.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-no-accept-numpy.'

{'insert_models': {'returning': [{'models': [{'id': 1150}]}]}}

In [79]:
model_no_accept_numpy = response['insert_models']['returning'][0]['models'][0]['id']

In [80]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_no_accept_numpy
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-no-accept-numpy',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-no-accept-numpy.zip/model'}}},
   'id': 1150,
   'image_path': None,
   'status': 'error',
   'task_id': '4593aaaa-79f2-47ce-944f-ba280ca2f273',
   'file_info': {'version': 'f806776c-33e2-4980-8a61-768ced2b6f06',
    'sha': '492c38a89d0f91d91c6bac0f5a267c567965ff7e58a16e9cc9f337ec726fb65c',
    'file_name': './models/byop-no-accept-numpy.zip',
    'size': 54694520},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:12:12.242968+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1721,
   'model_version_id': 1150,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   '

### Test 7: A Predict method that does not return a dictionary of numpy arrays

In [81]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-no-return-numpy'
model_file_name = './models/byop-no-return-numpy.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-no-return-numpy.'

{'insert_models': {'returning': [{'models': [{'id': 1151}]}]}}

In [82]:
model_no_return_numpy_id = response['insert_models']['returning'][0]['models'][0]['id']

In [83]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_no_return_numpy_id
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-no-return-numpy',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-no-return-numpy.zip/model'}}},
   'id': 1151,
   'image_path': None,
   'status': 'error',
   'task_id': '93934cd1-5fcf-4b3d-8be2-6d0327eccc4c',
   'file_info': {'version': '25e5d6c3-b5d0-4aca-9c60-003d4be397c0',
    'sha': 'b2fdd794d4f057d19daece32121f59c12f98854d75bd57b2bea3e54dbd90b4da',
    'file_name': './models/byop-no-return-numpy.zip',
    'size': 54694513},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:15:48.888926+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1722,
   'model_version_id': 1151,
   'runtime': 'flight',
   'filter_threshold': None,
   'tensor_fields': None,
   '

###  Test 8: Multiple classes inherit Inference class



In [84]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-vgg16-multiple-inference'
model_file_name = './models/byop-llamacpp-infbuild-multi-inherit.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-vgg16-multiple-inference.'

{'insert_models': {'returning': [{'models': [{'id': 1152}]}]}}

In [85]:
model_multiple_inference = response['insert_models']['returning'][0]['models'][0]['id']

In [86]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": model_multiple_inference
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-vgg16-multiple-inference',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-infbuild-multi-inherit.zip/model'}}},
   'id': 1152,
   'image_path': None,
   'status': 'error',
   'task_id': 'eb1718a8-7144-438c-9f78-e201eeddb1af',
   'file_info': {'version': '675c66ba-2a54-484d-8c4f-abcfb90a2a8e',
    'sha': 'efaba3fa2a0fc40b3f8e40639401be13f617ffadddb17dcc45586b302bdade03',
    'file_name': './models/byop-llamacpp-infbuild-multi-inherit.zip',
    'size': 1090659},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:17:40.258136+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1723,
   'model_version_id': 1152,
   'runtime': 'flight',
   'filter_threshol

###  Test 9: A .py file without an InferenceBuilder class

In [87]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-vgg16-missing-inference-builder'
model_file_name = './models/byop-llamacpp-no-inf-builder.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-vgg16-missing-inference-builder.'

{'insert_models': {'returning': [{'models': [{'id': 1155}]}]}}

In [88]:
byop_missing_inference_builder_id = response['insert_models']['returning'][0]['models'][0]['id']

In [89]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_inference_builder_id
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

...

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-vgg16-missing-inference-builder',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-no-inf-builder.zip/model'}}},
   'id': 1155,
   'image_path': None,
   'status': 'error',
   'task_id': '94a166ca-3888-486f-a85f-b99d0b675b38',
   'file_info': {'version': 'eca75c36-364e-4299-a5a6-a44162db7119',
    'sha': '63e36a6c0956888bcc955758bfc9ffcc4ee8e3415b8c060750a715f2d3534ecd',
    'file_name': './models/byop-llamacpp-no-inf-builder.zip',
    'size': 1090375},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:27:21.657296+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1726,
   'model_version_id': 1155,
   'runtime': 'flight',
   'filter_threshold': None,

###  Test 10: An InferenceBuilder class without an inference method

In [90]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-missing-inference-method'
model_file_name = './models/byop-llamacpp-infbuild-no-inf.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-missing-inference-method.'

{'insert_models': {'returning': [{'models': [{'id': 1156}]}]}}

In [91]:
byop_missing_inference_method_id = response['insert_models']['returning'][0]['models'][0]['id']

In [92]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_inference_method_id
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-missing-inference-method',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-infbuild-no-inf.zip/model'}}},
   'id': 1156,
   'image_path': None,
   'status': 'error',
   'task_id': '6ee88853-787b-4fe2-8426-239a3a0d81f4',
   'file_info': {'version': '8c481f4d-b42a-40e1-92be-6dd5df648624',
    'sha': 'f8d24672bb026a7d2cca8a94bbdfb58cc99b631ab6238bd3f27969578381fc29',
    'file_name': './models/byop-llamacpp-infbuild-no-inf.zip',
    'size': 1090539},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:30:41.214986+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1727,
   'model_version_id': 1156,
   'runtime': 'flight',
   'filter_threshold': None,
   '

###  Test 11: An InferenceBuilder class without a create method

In [93]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-missing-create-method'
model_file_name = './models/byop-llamacpp-infbuild-no-create.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-missing-create-method.'

{'insert_models': {'returning': [{'models': [{'id': 1157}]}]}}

In [94]:
byop_missing_create_method_id = response['insert_models']['returning'][0]['models'][0]['id']

In [95]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_missing_create_method_id
}

status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

...

'attempting_load_container'

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-missing-create-method',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-llamacpp-infbuild-no-create.zip/model'}}},
   'id': 1157,
   'image_path': None,
   'status': 'error',
   'task_id': 'c0a0a449-65c0-4fde-ab95-d8bb9554657c',
   'file_info': {'version': '1ea208a4-5c90-4e9f-906b-f3d70a99c069',
    'sha': 'c17eaa8d111f5fb513b832c2a2877b375eb8662c140c961445462baa8b0629a3',
    'file_name': './models/byop-llamacpp-infbuild-no-create.zip',
    'size': 1090544},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:32:44.557519+00:00',
   'deployed': False,
   'error_summary': 'null'},
  'config': {'id': 1728,
   'model_version_id': 1157,
   'runtime': 'flight',
   'filter_threshold': None,
 

### Test 12: Package dependencies fail to install

In [96]:
endpoint = f"{wl.api_endpoint}/v1/api/models/upload_and_convert"

model_name = 'byop-bad-dependancies'
model_file_name = './models/byop-bad-dependancies.zip'

metadata = {
    "name": model_name,
    "visibility": "private",
    "workspace_id": workspace.id(),
    "conversion": {
        "framework": framework,
        "python_version": "3.8",
        "requirements": []
    },
    "input_schema": encoded_input_schema,
    "output_schema": encoded_output_schema,
}

headers = wl.auth.auth_header()

files = {
    'metadata': (None, json.dumps(metadata), "application/json"),
    'file': (model_file_name, open(model_file_name,'rb'),'application/octet-stream')
}

response = requests.post(endpoint, files=files, headers=headers).json()

display(f"Uploaded Model Name: {model_name}.")
display(response)



'Uploaded Model Name: byop-bad-dependancies.'

{'insert_models': {'returning': [{'models': [{'id': 1159}]}]}}

In [97]:
byop_bad_dependancies_id = response['insert_models']['returning'][0]['models'][0]['id']

In [98]:
# Retrieve the token 
headers = wl.auth.auth_header()

endpoint = f"{wl.api_endpoint}/v1/api/models/get_version_by_id"

data = {
  "model_version_id": byop_bad_dependancies_id
}


status = "pending_load_container"

while (status != 'error'):
  time.sleep(15)
  response = requests.post(endpoint, json=data, headers=headers, verify=True).json()
  status = response['model_version']['model_version']['status']
  display(status)

display(response)

'attempting_load_container'

'attempting_load_container'

'error'

{'model_version': {'model_version': {'name': 'byop-bad-dependancies',
   'visibility': 'private',
   'workspace_id': 1814,
   'conversion': {'python_version': '3.8',
    'requirements': [],
    'framework': 'custom',
    'framework_config': {'framework': 'custom',
     'config': {'model_path': './models/byop-bad-dependancies.zip/model'}}},
   'id': 1159,
   'image_path': None,
   'status': 'error',
   'task_id': '23000b29-c78a-4089-b70a-5d2ef4c476e0',
   'file_info': {'version': '6ddc2951-cb6d-4266-aff4-4b3088a1a38e',
    'sha': 'd080aba4c2091fe6f3c252377c06b3eaed9726b2e6e48087fd4fb76f112f7b3c',
    'file_name': './models/byop-bad-dependancies.zip',
    'size': 54694526},
   'created_on_version': '2025.2.1',
   'created_by': 'john.hummel@wallaroo.ai',
   'created_at': '2025-12-23T22:37:10.834889+00:00',
   'deployed': False,
   'error_summary': 'Could not pip install user provided requirements.txt'},
  'config': {'id': 1730,
   'model_version_id': 1159,
   'runtime': 'flight',
   'filt